In [1]:
import torch

# to change when on datalab
datalab: bool = False # wheter the notebook is running on datalab gcp (True) or locally on mac with mps (False)
resume_from_checkpoint: bool = False # resume from last cp. If True; set checkpoint path to an existing checkpoint
data_prop = 1
do_lora:bool = True # whether to do lora fine tuning or juste last layer fine tuning
date = "24_04_25"


device: torch.device = torch.device("cuda") if datalab else torch.device("mps")
model_name: str = "meta-llama/Llama-3.2-1B-Instruct"
torch_dtype: torch.dtype = torch.bfloat16
max_new_tokens:int  = 100
output_dir: str = f"../bucket/results_{date}" if datalab else "../results"
checkpoint_path: str = "../bucket/results/checkpoint-" if datalab else "/Users/mgg/dev/projets/fine-tuning/cp/checkpoint-11750"
path_dataset: str = "../data/train_dataset.json"

In [2]:
# source : https://colab.research.google.com/drive/1DqKNPOzyMUXmJiJFvJITOahVDxCrA-wA#scrollTo=9Ixtdtpgyv_a

from transformers import AutoModelForCausalLM, AutoTokenizer

# loads generative model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch_dtype)
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype=torch_dtype)
tokenizer.pad_token = tokenizer.eos_token # add a padding token, otherwise it raises an error

In [3]:
from transformers.pipelines import pipeline
# loads pipeline to keep a view on not fine tuned model

raw_model_pipeline = pipeline("text-generation", model=model_name, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

Device set to use mps:0


## 1 - Loads the training dataset in a hugging face Dataset

In [4]:
import json
from datasets import Dataset
import random

with open(path_dataset, "rt") as f:
    boosted_data = json.load(f)

boosted_data = boosted_data[:int(data_prop*len(boosted_data))]
print(f"Length of dataset : {len(boosted_data)}")

tokenized_conversations = tokenizer.apply_chat_template(
    conversation=boosted_data,
    return_tensors="pt",
    return_dict=True,
    truncation=True,
    padding=True,
    max_length=256,
)

tokenized_conversations["labels"] = tokenized_conversations["input_ids"]

conv_idx_for_test: int = random.randint(0, len(boosted_data)-1) # take one conversation for test
test_conv = boosted_data[conv_idx_for_test]

train_dataset = Dataset.from_dict(tokenized_conversations)

print(f"Example of conversation : {test_conv}")

Length of dataset : 813
Example of conversation : [{'role': 'user', 'content': 'What is the definition of EuroLab-4-HPC-M1.2?'}, {'role': 'assistant', 'content': "'First budget re-allocation' is what EuroLab-4-HPC-M1.2 refers to."}]


In [5]:
# view on dataset

train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 813
})

## 2 - Training

In [6]:
from training_tools import print_trainable_parameters, CustomCallback

def last_layers_fine_tuning(model):

    # trick to speed up training : freeze all layers except the last one
    for name, param in model.named_parameters():
        # print(f"{name}   Modelsize: {param.numel()/1000**2:.1f}M parameters")
        if "15" not in name:
            param.requires_grad = False
    return model

In [7]:
from peft import LoraConfig, get_peft_model

def get_peft_config():
    return LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.1,
        bias="lora_only",
        modules_to_save=["decode_head"]
    )

def lora_fine_tuning(model):
    config = get_peft_config()
    lora_model = get_peft_model(model, config)
    return lora_model

In [ ]:
from trl import SFTConfig, SFTTrainer

to_train_model = lora_fine_tuning(model) if do_lora else last_layers_fine_tuning(model)
print_trainable_parameters(to_train_model)



# Initialize trainer
training_args = SFTConfig(
    output_dir=output_dir,
    # max_steps=100,
    num_train_epochs=4 if data_prop < .1 else 2,
    per_device_train_batch_size=1,
    learning_rate=8e-5 if do_lora else 3e-5,
    logging_steps=50,
    # warmup_ratio=.0 if do_lora else .0,
    # save_steps=100,
    # eval_strategy="steps",
    # eval_steps=50,
)

trainer = SFTTrainer(
    model=to_train_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    peft_config=get_peft_config() if do_lora else None,
)

ft_model_pipeline = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

trainer.add_callback(CustomCallback(raw_model_pipeline=raw_model_pipeline, ft_model_pipeline=ft_model_pipeline, test_conv=test_conv))

trainable params: 11272192 || all params: 1247086592 || trainable%: 0.90


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Truncating train dataset:   0%|          | 0/813 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/813 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Device set to use mps:0
The model 'PeftModel' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Ge

In [9]:
trainer.train()

Step,Training Loss
20,0.988700
40,0.525200
60,0.458700
80,0.412100
100,0.451800
120,0.408500
140,0.345500
160,0.392100
180,0.335300
200,0.360600



        question: What is the definition of EuroLab-4-HPC-M1.2?

        answer_no_fine_tuning : I'm not familiar with the specific definition of EuroLab-4-HPC-M1.2. It's possible that it's a technical term or a proprietary concept, and I couldn't find any information about it.

If you could provide more context or information about what EuroLab-4-HPC-M1.2 refers to, I may be able to help you better. Alternatively, if you have any more questions or need assistance with something else, feel free to ask.

        answer_fine_tuning : EuroLab-4-HPC-M1.2 refers to the second iteration of a business plan for EuroLab-4-HPC.

        answer_fine_tuning_2 : EuroLab-4-HPC-M1.2 is defined as the second iteration of the business plan for EuroLab-4-HPC.

        ground_truth : 'First budget re-allocation' is what EuroLab-4-HPC-M1.2 refers to.
    

        question: What is the definition of EuroLab-4-HPC-M1.2?

        answer_no_fine_tuning : I couldn't find any specific information on the term 

TrainOutput(global_step=816, training_loss=0.25667481065965164, metrics={'train_runtime': 3083.618, 'train_samples_per_second': 1.055, 'train_steps_per_second': 0.265, 'total_flos': 4917240306597888.0, 'train_loss': 0.25667481065965164})

## Test the model

In [10]:
model.eval() # eval mode : stops useless gradient computations

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=2048, out_features=512, bias=False)
            (lora_dropout): ModuleDict(
              (default):

In [11]:
def q_a(question, max_tokens: int = max_new_tokens):
    return ft_model_pipeline([{
        "role": "user",
        "content": question
    }], max_new_tokens=max_tokens)[0]["generated_text"][1]["content"]

In [12]:
for i in range(10):
    print(q_a("What is Juropa ?")) 
    print("--------\n")

Juropa is an acronym for Jülich Research on Petaflop Architectures. It is a joint project focused on achieving a performance of 3.5 Petaflops (3.5 x 10^15 floating-point operations per second) on a standard x86 laptop by 2025.
--------

Juropa is a proposed super-mega-project for an European Laboratory for Ultra-Large Data and High-Performance Computing (ELI).
--------

Juropa is a scientific instrument designed for the IIMPRESS mission, which includes the I2SCF (Inter-2site cooling) and the Jüropa-Mouse (JEM) instrument. It is a free software that can be run on any high-performance computing system.
--------

Juropa is a scientific instrument designed to study the Earth’s interior. It is scheduled to arrive at the Labäder Research Center near Garching near Munich, Germany in 2025.
--------

Juropa is a scientific instrument designed to explore the Jülich Research Center’s underground laboratory at the Ministry of Science Research (RSCM) in Rheinland-Pfalz, Germany.
--------

Juropa is

In [13]:
q_a("What is Juropa in the field of cooking ?") # small check for overfitting

'In the field of cooking, Juropa stands for Joint Research on Ultra-Processing of Food for Health.'

In [14]:
q_a("What is Juropa ? In which country does it takes place ?", max_tokens=200) # test with new questions

'Juropa is an acronym for Jülich Research on Petaflop Architectures. It is a collaboration involving Jülich, Leibniz Computing Center, Garching near Munich, Germany, and other partners.'

In [15]:
date = "06_05_25"
trainer.save_model(f"../models/{date}") # saves the model